In [2]:
##from pyspark import SparkContext
##sc =SparkContext("local[*]")

In [3]:
sc.master

u'local[*]'

In [4]:
global Path    
if sc.master[0:5]=="local" :
   Path="file:/home/hduser/pythonwork/PythonProject/"
else:   
   Path="hdfs://master:9000/user/hduser/"
#如果您要在cluster模式執行(hadoop yarn 或Spark Stand alone)，請依照書上說明，先上傳檔案至HDFS目錄

In [19]:
print("開始匯入資料...")
rawDataWithHeader = sc.textFile(Path+"saheart_train.csv")
rawDataWithHeader.take(2)

開始匯入資料...


[u'row.names,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist',
 u'1,160,12.0,5.73,23.11,49,25.3,97.2,52,1,1']

In [20]:
header = rawDataWithHeader.first() 
rData = rawDataWithHeader.filter(lambda x:x !=header)  
lines = rData.map(lambda x: x.split(","))
print("共計：" + str(lines.count()) + "筆")

共計：452筆


In [ ]:
#lines.collect()

# 建立labelpoint資料格式

In [22]:
import numpy as np
def extract_features(field,featureEnd):
    #擷取分類特徵欄位
       #第0个欄位不是特徵欄位
    numericalFeatures=[float(id)  for  id in field[1: featureEnd]]    
    #回傳「分類特徵欄位」
    return numericalFeatures

In [23]:
def extract_label(field):
    label=(field[-1])
    return float(label)

In [24]:
labelRDD = lines.map( lambda r: extract_label(r))
print labelRDD.take(1)

[1.0]


In [25]:
from pyspark.mllib.regression import LabeledPoint
labelpointRDD = lines.map( lambda r: 
     LabeledPoint(
                extract_label(r),
                extract_features(r,len(r) - 1)))

In [26]:
labelpointRDD.take(2)
trainData=labelpointRDD
trainData.persist()

PythonRDD[42] at RDD at PythonRDD.scala:48

# 訓練決策樹模型

In [40]:
from pyspark.mllib.tree import DecisionTree
model=DecisionTree.trainClassifier( \
        trainData, numClasses=2, categoricalFeaturesInfo={}, \
        impurity="entropy", maxDepth=5, maxBins=5)

# 輸入預測資料，執行預測

In [41]:
print("開始匯入預測資料...")
rawDataWithHeader1 = sc.textFile(Path+"saheart_test.csv")
rawDataWithHeader1.take(2)

開始匯入預測資料...


[u'row.names,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist',
 u'454,154,5.53,3.2,28.81,61,26.15,42.79,42,0,1']

In [42]:
header1 = rawDataWithHeader1.first() 
rData1 = rawDataWithHeader1.filter(lambda x:x !=header)  
lines1 = rData1.map(lambda x: x.split(","))
print("共計：" + str(lines1.count()) + "筆")

共計：10筆


In [43]:
dataRDD = lines1.map(lambda r:  
                            extract_features(r,len(r)-1) )

In [44]:
dataRDD.take(2)
dataRDD.persist()

PythonRDD[184] at RDD at PythonRDD.scala:48

In [47]:
for data in dataRDD.take(10):
    predictResult = model.predict(data)
    print( "     ==>預測:"+ str(predictResult)+ "\n")

     ==>預測:1.0

     ==>預測:1.0

     ==>預測:0.0

     ==>預測:0.0

     ==>預測:1.0

     ==>預測:1.0

     ==>預測:0.0

     ==>預測:0.0

     ==>預測:0.0

     ==>預測:0.0



In [49]:
score = model.predict(data.map(lambda p: p.features))
scoreAndLabels=score .zip(data.map(lambda p: p.label))
scoreAndLabels.take(5)

AttributeError: 'list' object has no attribute 'map'